In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/home/andreas/Desktop/Cyberlens/Pre-interview_assessment/vulnerable_robot_challenge.csv")

In [3]:
normal = df.loc[df['flag'] == 0, :]
normal.describe()

,t,CPU,RxKBTot,TxKBTot,WriteKBTot,Watts,Amps,RMS,diff_encoder_l,flag
count,4132.000000,4132.000000,4132.000000,4132.000000,4132.000000,4132.000000,4132.000000,4132.000000,4132.000000,4132.0
mean,171.991452,23.350803,244.917846,14.778896,0.923035,89.733106,0.566935,0.158382,17.305770,0.0
std,33.230427,9.146263,1353.260313,77.508104,1.511957,9.041810,0.059048,0.087423,8.762966,0.0
min,112.220000,0.000000,0.000000,0.000000,0.000000,62.700000,0.401004,0.008099,0.000012,0.0
25%,148.580000,26.000000,1.000000,0.960000,0.000000,84.000000,0.536825,0.106521,11.200000,0.0
50%,164.970000,26.800000,1.000000,1.000000,0.000000,92.700000,0.587531,0.148595,18.000000,0.0
75%,199.645000,27.000000,1.000000,1.000000,1.520000,98.176835,0.614895,0.196046,22.943282,0.0
max,228.780000,42.000000,8801.000000,506.000000,10.000000,101.199602,0.635988,1.136100,89.718974,0.0


In [4]:
fraud = df.loc[df['flag'] == 1, :]
fraud.describe()

,t,CPU,RxKBTot,TxKBTot,WriteKBTot,Watts,Amps,RMS,diff_encoder_l,flag
count,4804.000000,4804.000000,4804.000000,4804.000000,4804.000000,4804.000000,4804.000000,4804.000000,4804.000000,4804.0
mean,173.063281,18.942756,4333.229184,244.369484,1.369172,87.776590,0.566919,0.159151,14.446373,1.0
std,25.462097,7.969489,4321.163783,244.466168,1.907361,7.370592,0.058446,0.137175,8.305075,0.0
min,122.000000,1.000000,0.000000,0.000000,0.000000,62.575296,0.400111,0.008062,0.000000,1.0
25%,157.995000,11.240000,0.960000,0.000000,0.000000,83.900000,0.542000,0.080659,10.396769,1.0
50%,177.000000,12.870000,2892.570000,119.340000,0.560000,92.479720,0.592000,0.126455,13.000000,1.0
75%,192.005000,27.000000,8695.620000,498.280000,2.160000,93.200000,0.603124,0.187214,20.333341,1.0
max,215.000000,41.000000,8810.000000,508.000000,11.000000,98.743706,0.631980,1.817189,84.166746,1.0


In [5]:
normal.shape

(4132, 10)

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize
columns = [1, 2, 3, 5]

Data_norm = MinMaxScaler().fit_transform(df.iloc[:,:-1])
Data_norm = pd.DataFrame(Data_norm)
Data_norm['flag'] = df['flag']
normal = Data_norm.loc[Data_norm['flag'] == 0, :]
fraud = Data_norm.loc[Data_norm['flag'] == 1, :]

# Data = Data_norm.sample(frac = .2, random_state = 42)
X = normal.loc[:,columns]
y = normal['flag']
X_at = fraud.loc[:,columns]
print(X.shape)
print(y.shape)

(4132, 4)
(4132,)


In [13]:
test_this = X_at.iloc[[2]]

In [14]:
test_this

,1,2,3,5
491,0.642857,0.000114,0.001969,0.549517


In [16]:
from sklearn.model_selection import train_test_split

x_normal_train, x_normal_test = train_test_split(
    X, test_size=0.25, random_state=42)

In [17]:

print(f"Normal train count: {len(x_normal_train)}")
print(f"Normal test count: {len(x_normal_test)}")

Normal train count: 3099
Normal test count: 1033


In [18]:
from sklearn import metrics
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [19]:
model = Sequential()
model.add(Dense(25, input_dim=X.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(X.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()
model.fit(x_normal_train,x_normal_train,verbose=1,epochs=100)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25)                125       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 78        
_________________________________________________________________
dense_2 (Dense)              (None, 25)                100       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 104       
Total params: 407
Trainable params: 407
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
97/97 [==============================] - 0s 954us/step - loss: 0.1012
Epoch 2/100
97/97 [==============================] - 0s 894us/step - loss: 0.0102
Epoch 3/100
97/97 [==============================] - 0s 870us/step - loss: 0.0028
Epoch 4/10

In [20]:
pred1 = model.predict(x_normal_test)
score1 = np.sqrt(metrics.mean_squared_error(pred1,x_normal_test))


pred2 = model.predict(X)
score2 = np.sqrt(metrics.mean_squared_error(pred2,X))


pred3 = model.predict(X_at)
score3 = np.sqrt(metrics.mean_squared_error(pred3,X_at))


print(f"Out of Sample Score (RMSE): {score1}")
print(f"Insample Normal Score (RMSE): {score2}")
print(f"Attack Underway Score (RMSE): {score3}")

Out of Sample Score (RMSE): 0.0009896291869394512
Insample Normal Score (RMSE): 0.000981324502080185
Attack Underway Score (RMSE): 0.019791063111534467


In [21]:
threshold = np.max(score1)
print("Reconstruction error threshold: ", threshold)

Reconstruction error threshold:  0.0009896291869394512


In [22]:
def check_status(prediction):
    if prediction > threshold:
        print("anomaly")
    else:
        print("we are doing good")

In [28]:
score_test = model.predict(test_this)
score_test1 = np.sqrt(metrics.mean_squared_error(score_test,test_this))
score_test1

0.0005611648567257975

In [27]:
check_status(score_test1)

we are doing good


In [45]:
x_normal_test[1]

5266    0.423810
2161    0.642857
5326    0.214286
2703    0.023810
2046    0.646190
          ...   
8706    0.638095
6025    0.629048
7519    0.655714
1651    0.642857
2061    0.653333
Name: 1, Length: 1033, dtype: float64

In [65]:
# plt.plot(X[1])
# plt.show()
# plt.plot(pred2[1])
# plt.show()

In [68]:
score1 = score1.reshape((-1))
score1


array([0.00544834])

In [70]:
anomalies = (score1 > threshold).tolist()
print("Number of anomaly samples: ", np.sum(anomalies))
print("Indices of anomaly samples: ", np.where(anomalies))

Number of anomaly samples:  1
Indices of anomaly samples:  (array([0]),)
